In [ ]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# default_exp repositories

<IPython.core.display.Javascript object>

In [ ]:
from will_it_saturate.repositories import BaseRepository

<IPython.core.display.Javascript object>

In [ ]:
# export

import json
import sqlite3


def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d


class SqliteRepository(BaseRepository):
    connection: sqlite3.Connection

    class Config:
        arbitrary_types_allowed = True

    def execute_stmt(self, stmt, ignore_error=False):
        cursor = self.connection.cursor()
        try:
            cursor.execute(stmt)
        except sqlite3.OperationalError as exc:
            if not ignore_error:
                raise (exc)

    def create_benchmark_table(self):
        stmt = """
            create table benchmark (
                benchmark_id INTEGER PRIMARY KEY,
                created DATE DEFAULT (datetime('now','localtime')),
                machine_id TEXT NOT NULL UNIQUE,
                uname TEXT NOT NULL,
                data TEXT NOT NULL
            )
        """
        self.execute_stmt(stmt, ignore_error=True)

    def create_host_table(self):
        stmt = """
            create table host (
                host_id INTEGER PRIMARY KEY,
                created DATE DEFAULT (datetime('now','localtime')),
                machine_id TEXT NOT NULL UNIQUE,
                data TEXT NOT NULL
            )
        """
        self.execute_stmt(stmt, ignore_error=True)

    def create_result_table(self):
        stmt = """
            create table result (
                result_id INTEGER PRIMARY KEY,
                server_host_id integer not NULL,
                client_host_id integer not NULL,
                created DATE DEFAULT (datetime('now','localtime')),
                server text not NULL,
                client text not NULL,
                file_size integer not NULL,
                complete_size integer not NULL,
                elapsed real not NULL,
                FOREIGN KEY(server_host_id) REFERENCES host(host_id)
                FOREIGN KEY(client_host_id) REFERENCES host(host_id)
            )
        """
        self.execute_stmt(stmt, ignore_error=True)

    def create_tables(self):
        self.create_host_table()
        self.create_result_table()

    def add_benchmark(self, benchmark):
        stmt = """
            insert into benchmark (machine_id, uname, data)
            values (?, ?, ?)
        """
        cursor = self.connection.cursor()
        machine_id = benchmark.machine_id
        uname = benchmark.uname_json
        data = benchmark.json()
        try:
            cursor = cursor.execute(stmt, [machine_id, uname, data])
        except sqlite3.IntegrityError as e:
            pass
        self.connection.commit()

    def get_benchmark(self, benchmark):
        stmt = """
            select *
              from benchmark
             where machine_id=?
        """
        cursor = self.connection.cursor()
        cursor.execute(stmt, [benchmark.machine_id])
        return cursor.fetchone()

    def get_benchmark_id(self, benchmark):
        row = self.get_benchmark(benchmark)
        if row is None:
            return row
        return row[0]

    def get_machine_to_host_id(self, machine_ids):
        question_marks = ", ".join(["?" for _ in machine_ids])
        stmt = f"""
            select host_id, machine_id
              from host
             where machine_id in ({question_marks})
        """
        cursor = self.connection.cursor()
        cursor.execute(stmt, machine_ids)
        rows = cursor.fetchall()
        return {row["machine_id"]: row["host_id"] for row in rows}

    def insert_host_details(self, host_details):
        stmt = """
            insert into host (machine_id, data)
            values (?, ?)
        """
        cursor = self.connection.cursor()
        try:
            cursor = cursor.execute(
                stmt, [host_details.machine_id, host_details.json()]
            )
        except sqlite3.IntegrityError as e:
            pass
        self.connection.commit()
        return cursor.rowcount

    def get_result(self, benchmark, result):
        benchmark_id = self.get_benchmark_id(benchmark)
        if benchmark_id is None:
            self.add_benchmark(benchmark)
        benchmark_id = self.get_benchmark_id(benchmark)
        assert benchmark_id is not None
        stmt = """
            select *
              from result
             where benchmark_id=?
               and server=?
               and client=?
               and file_size=?
               and complete_size=?
        """
        cursor = self.connection.cursor()
        cursor.execute(
            stmt,
            [
                benchmark_id,
                result.server,
                result.client,
                result.file_size,
                result.complete_size,
            ],
        )
        row = cursor.fetchone()
        if row is None:
            return result
        result.elapsed = row[7]
        return result

    def get_results(self):
        stmt = """
            select *
              from result
        """
        cursor = self.connection.cursor()
        cursor.execute(stmt)
        rows = cursor.fetchall()
        return rows

    def get_or_create_hosts_from_result(self, result):
        machine_id_to_server_details = {
            result.server_details.machine_id: result.server_details,
            result.client_details.machine_id: result.client_details,
        }
        machine_ids = list(machine_id_to_server_details.keys())
        machine_to_host_id = self.get_machine_to_host_id(machine_ids)
        for machine_id, host_detail in machine_id_to_server_details.items():
            if machine_id not in machine_to_host_id:
                self.insert_host_details(host_detail)
        return self.get_machine_to_host_id(machine_ids)

    def add_result(self, result):
        machine_to_host_id = self.get_or_create_hosts_from_result(result)
        result_row = {
            "server_host_id": machine_to_host_id[result.server_details.machine_id],
            "client_host_id": machine_to_host_id[result.client_details.machine_id],
            "server": result.server.json(),
            "client": result.client.json(),
            "file_size": result.file_size,
            "complete_size": result.complete_size,
            "elapsed": result.elapsed,
        }
        columns = list(result_row.keys())
        columns_str = ",".join(columns)
        values_str = ",".join(["?" for c in columns])

        stmt = f"""
            insert into result ({columns_str})
            values ({values_str})
        """
        cursor = self.connection.cursor()

        values = [result_row[c] for c in columns]
        try:
            cursor = cursor.execute(stmt, values)
        except sqlite3.IntegrityError as e:
            pass
        self.connection.commit()
        return cursor.rowcount

    @classmethod
    def build_repository(cls, database_name):
        conn = sqlite3.connect(database_name)
        conn.row_factory = dict_factory
        repo = cls(connection=conn)
        repo.create_tables()
        return repo

<IPython.core.display.Javascript object>

## Tests

In [ ]:
from will_it_saturate.results import Result
from will_it_saturate.hosts import HostDetails

<IPython.core.display.Javascript object>

### Add Host to Repository

In [ ]:
repo = SqliteRepository.build_repository(":memory:")
machine_to_host_id = repo.get_machine_to_host_id(["asdf"])
assert len(machine_to_host_id) == 0

<IPython.core.display.Javascript object>

### Add HostDetails to Repository

In [ ]:
machine_id1, cpuinfo = "asdf", {"arch": "X86_64"}
host_detail = HostDetails(machine_id=machine_id1, cpuinfo=cpuinfo)

rowcount = repo.insert_host_details(host_detail)
assert rowcount == 1

machine_to_host_id = repo.get_machine_to_host_id([machine_id1])
assert machine_to_host_id == {machine_id1: 1}

<IPython.core.display.Javascript object>

### Add two HostDetails to Repository with one Already Existing

In [ ]:
machine_id2 = "bsdf"
host_detail = HostDetails(machine_id=machine_id2, cpuinfo=cpuinfo)

rowcount = repo.insert_host_details(host_detail)
assert rowcount == 1

machine_to_host_id = repo.get_machine_to_host_id([machine_id1, machine_id2, "csdf"])
assert machine_to_host_id == {machine_id1: 1, machine_id2: 2}

<IPython.core.display.Javascript object>

### Add all Hosts Needed for Result

In [ ]:
dummy_result_data = {
    "server": {"name": "server"},
    "client": {"name": "client"},
    "file_size": 10,
    "complete_size": 100,
}

kwargs = {
    **dummy_result_data,
    "server_details": HostDetails(machine_id="dsdf", cpuinfo={}),
    "client_details": HostDetails(machine_id="esdf", cpuinfo={}),
}
result = Result(**kwargs)
machine_to_host_id = repo.get_or_create_hosts_from_result(result)
assert machine_to_host_id == {
    result.server_details.machine_id: 3,
    result.client_details.machine_id: 4,
}

<IPython.core.display.Javascript object>

### Add a Complete Result to Repository

In [ ]:
from will_it_saturate.results import Result
from will_it_saturate.servers import BaseServer
from will_it_saturate.clients import BaseClient


server = BaseServer(name="server", host="127.0.0.1", port=8000)
client = BaseClient(name="fake client")
server_details = HostDetails(machine_id="server_1", cpuinfo={"foo": "bar"})
client_details = HostDetails(machine_id="client_1", cpuinfo={"bar": "baz"})
result = Result(
    server=server,
    client=client,
    server_details=server_details,
    client_details=client_details,
    file_size=10 ** 5,
    complete_size=10 ** 8,
    elapsed=400,
)
inserted_rows = repo.add_result(result)

<IPython.core.display.Javascript object>

### Get Results from Repository

In [ ]:
results = repo.get_results()
print(results)

[{'result_id': 1, 'server_host_id': 5, 'client_host_id': 6, 'created': '2021-04-21 08:08:35', 'server': '{"protocol": "http", "name": "server", "host": "127.0.0.1", "port": 8000}', 'client': '{"name": "fake client"}', 'file_size': 100000, 'complete_size': 100000000, 'elapsed': 400.0}]


<IPython.core.display.Javascript object>

## Usage

In [ ]:
# dont_test

repo = SqliteRepository.build_repository(":memory:")

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

from will_it_saturate.results import Result

server = BaseServer(name="server", host="127.0.0.1", port=8000)
client = BaseClient(name="fake client")
server_details = HostDetails(machine_id="server_1", cpuinfo={"foo": "bar"})
client_details = HostDetails(machine_id="client_1", cpuinfo={"bar": "baz"})
result = Result(
    server=server,
    client=client,
    server_details=server_details,
    client_details=client_details,
    file_size=10 ** 5,
    complete_size=10 ** 8,
    elapsed=400,
)
inserted_rows = repo.add_result(result)

<IPython.core.display.Javascript object>

In [ ]:
# repo.get_benchmark(benchmark)

<IPython.core.display.Javascript object>

In [ ]:
# benchmark.json()

<IPython.core.display.Javascript object>

In [ ]:
# repo.get_benchmark(benchmark)

<IPython.core.display.Javascript object>

In [ ]:
# benchmark.machine_id = "foobar"

<IPython.core.display.Javascript object>

In [ ]:
# file_size = 10 ** 6
# complete_size = 100 * file_size
# result = Result(
#     server="nginx",
#     client="httpx",
#     file_size=file_size,
#     complete_size=complete_size,
# )

# result = repo.get_result(benchmark, result)
# print(result.elapsed)

<IPython.core.display.Javascript object>

In [ ]:
# from will_it_saturate.core import Result

# file_size = 10 ** 6
# complete_size = 100 * file_size
# result = Result(
#     server="nginx",
#     client="httpx",
#     file_size=file_size,
#     elapsed=3.0,
#     complete_size=complete_size,
# )
# print(result.dict())

<IPython.core.display.Javascript object>

In [ ]:
# repo.add_result(benchmark, result)

<IPython.core.display.Javascript object>

# Export

In [ ]:
# dont_test

from nbdev.export import notebook2script

notebook2script()

Converted 00_host.ipynb.
Converted 01_django_views.ipynb.
Converted 01_fastapi_views.ipynb.
Converted 01_file.ipynb.
Converted 05_epochs.ipynb.
Converted 06_results.ipynb.
Converted 25_core.ipynb.
Converted 27_control_server.ipynb.
Converted 28_control_client.ipynb.
Converted 29_control_cli.ipynb.
Converted 30_servers.ipynb.
Converted 31_local_servers.ipynb.
Converted 32_docker_servers.ipynb.
Converted 40_clients.ipynb.
Converted 41_benchmark_clients.ipynb.
Converted 70_repositories.ipynb.
Converted 71_sqlite_repository.ipynb.
Converted 90_benchmark_without_benchmark.ipynb.
Converted 98_run_benchmark.ipynb.
Converted index.ipynb.


<IPython.core.display.Javascript object>